SINTAXIS DEL ETL

In [1]:
#Importe de librerías a usar
import gzip
import pandas as pd
import json
import ast

ETL ARCHIVO user_items

In [ ]:
#Cargue del archivo json y conversión a DT
def l_a_c():
    with gzip.open('users_items.json.gz','rt', encoding='utf-8') as file:
        return[ast.literal_eval(line.strip())
            for line in file]
        
datos = l_a_c()

df_item = pd.DataFrame(datos)

df_item.info()

In [ ]:
#Desanidar items

def de_nest_items(df):
    items = df['items'].explode()
    df = df.drop('items', axis=1)
    df = df.join(items.apply(pd.Series))
    return df

In [ ]:
df_item = de_nest_items(df_item.copy())

In [ ]:
#Se eliminan columnas innecesarias
df_item.drop([0,'steam_id','playtime_2weeks'], axis=1, inplace=True)

In [ ]:
#Se eliminan registros con valor nulo en todos los campos
df_item.dropna(how='all', inplace=True)

In [ ]:
#Se eliminan duplicados
df_item=df_item.drop_duplicates()

In [ ]:
#Se genera una muestra del dataframe para probar las funciones
df_item_sample = df_item.sample(50000)

In [5]:
df_item_sample

,user_id,items_count,user_url,item_id,item_name,playtime_forever
0,76561197976498814,4,http://steamcommunity.com/profiles/76561197976...,220,Half-Life 2,142.0
1,76561198064876158,47,http://steamcommunity.com/profiles/76561198064...,206420,Saints Row IV,1792.0
2,76561198019443690,60,http://steamcommunity.com/profiles/76561198019...,244850,Space Engineers,7737.0
3,BlueDragonOfficial,308,http://steamcommunity.com/id/BlueDragonOfficial,282440,Quake Live,0.0
4,Pootisman64,24,http://steamcommunity.com/id/Pootisman64,223530,Left 4 Dead 2 Beta,0.0
...,...,...,...,...,...,...
49995,76561198070981191,18,http://steamcommunity.com/profiles/76561198070...,316010,Magic Duels,32.0
49996,shockwave760,110,http://steamcommunity.com/id/shockwave760,458280,CS:GO Player Profiles: Happy - Team EnVyUs,0.0
49997,76561197960963799,98,http://steamcommunity.com/profiles/76561197960...,238960,Path of Exile,15099.0
49998,WaffyButt,88,http://steamcommunity.com/id/WaffyButt,280,Half-Life: Source,0.0


In [ ]:
#Se guarda la muestra eñ un archivo parquet
df_item_sample = df_item_sample.to_parquet("df_items_sample.parquet", index=False)

ETL ARCHIVO user_reviews

In [ ]:
#Cargue de las librerías a trabajar
import gzip
import pandas as pd
import json
import ast
import re


In [ ]:
#Convertir en dataframne el archivo json
def l_a_c():
    with gzip.open('user_reviews.json.gz','rt', encoding='utf-8') as file:
        return[ast.literal_eval(line.strip())
            for line in file]
        
datos = l_a_c()

df_user = pd.DataFrame(datos)

df_user

In [ ]:
#Se importa la librería a usar (posterior a la instalación del paquete TextBlob) para realizar el análisis de sentimientos
from textblob import TextBlob

In [ ]:
#Se crea la función para realizar el análisis de sentimientos
def sentiment_analysis(review):
    if isinstance(review, list) and len(review) > 0:
        text = review[0].get('review', '')
        sentiment = TextBlob(text).sentiment.polarity
        
        if sentiment < -0.2:
            return 0 # Negativo
        elif sentiment >= -0.2 and sentiment <= -0.2:
            return 1 # Neutral
        else:
            return 2 # Positivo
    else:
        return 1 # Valor para casos en que la reseña escrita este ausente

In [ ]:
#Se aplica la función creada a la coliumna reviwes, para realizar el análisis y generaer la columna solicitada Sentiment_analysis
df_user['sentiment_analysis'] = df_user['reviews'].apply(sentiment_analysis)

In [ ]:
#Proceso para desanidar reviews
#primero creamos una lista vacía donde vamos a almacenar los datos de la columna 'reviews'
data_review = []

In [ ]:
#import pandas as pd

for index, fila in df_user.iterrows():
    for review in fila['reviews']:
        # Verificar si 'review' es un diccionario
        if isinstance(review, dict):
            data_review.append({
                "user_id": fila["user_id"],
                "user_url": fila["user_url"],
                "sentiment_analysis": fila["sentiment_analysis"],
                "funny": review.get("funny", ""),
                "posted": review.get("posted", ""),
                "last_edited": review.get("last_edited", ""),  
                "item_id": review.get("item_id", ""),
                "helpful": review.get("helpful", ""),
                "recommend": review.get("recommend", ""),
                "review": review.get("review", "")
            })

# Convertir la lista de datos en un nuevo DataFrame
df_user_new= pd.DataFrame(data_review)

# Mostrar el DataFrame resultante
df_user_new.head(5)

In [ ]:
#Se eliminan columnas innecesarias
df_user_new.drop(['helpful','funny','last_edited','review'], axis=1, inplace=True)

In [ ]:
#Eliminamos los puntos de la columna 'posted'
df_user_new['posted'] = df_user_new['posted'].str.replace('.', '')
df_user_new['posted'] = df_user_new['posted'].str.replace(',', '')
df_user_new['posted'] = df_user_new['posted'].str.replace('Posted', '')

In [ ]:
# Función para comprobar y transformar el formato de las fechas
def analizar_fechas(df, columna):
    for index, fecha in enumerate(df[columna]):
        # Usar una expresión regular para verificar el formato deseado (dd-mm-AAAA)
        if not re.match(r'\d{2}-\d{2}-\d{4}', fecha):
            # Si no cumple con el formato, transformarlo en el formato correcto
            try:
                fecha_obj = pd.to_datetime(fecha, errors='raise')
                fecha_transformada = fecha_obj.strftime('%d-%m-%Y')
                df.at[index, columna] = fecha_transformada
            except ValueError:
                # Si no se puede transformar en una fecha válida, dejarlo como está
                pass

# Llamar a la función para verificar y transformar las fechas
analizar_fechas(df_user_new, 'posted')

# Imprimir el DataFrame resultante
print(df_user_new['posted'])

In [ ]:
df_user_new['posted'] = pd.to_datetime(df_user_new['posted'], format='mixed', dayfirst=True, errors='coerce')

In [ ]:
# Convertir la columna 'posted' a tipo cadena (string) si no lo es
df_user_new['posted'] = df_user_new['posted'].astype(str)

# Verificar si todos los valores en 'posted' tienen el mismo formato (dd-mm-AAAA)
formato_esperado = r'\d{2}-\d{2}-\d{4}'  # Expresión regular para el formato dd-mm-AAAA

df_user_new.head(5)

In [8]:
#Se eliminan los registros de fecha con dato NaN
df_user_new=df_user_new.dropna(subset=['posted'])
df_user_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25458 entries, 0 to 25457
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          25458 non-null  int64 
 1   user_id             25458 non-null  object
 2   user_url            25458 non-null  object
 3   sentiment_analysis  25458 non-null  int64 
 4   posted              25458 non-null  object
 5   item_id             25458 non-null  int64 
 6   recommend           25458 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 1.4+ MB


In [ ]:
#Ahora vamosa transformar la columna 'recommend'
df_user_new['recommend'] = df_user_new['recommend'].replace({True:1, False:0})

In [ ]:
#Se eliminan duplicados
df_user_new=df_user_new.drop_duplicates(subset=['user_id'])

In [7]:
df_user_new.head(5)

,Unnamed: 0,user_id,user_url,sentiment_analysis,posted,item_id,recommend
0,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2,2011-11-05,1250,1
1,3,js41637,http://steamcommunity.com/id/js41637,2,2014-06-24,251610,1
2,6,evcentric,http://steamcommunity.com/id/evcentric,2,NaT,248820,1
3,12,doctr,http://steamcommunity.com/id/doctr,2,2013-10-14,250320,1
4,18,maplemage,http://steamcommunity.com/id/maplemage,2,2014-04-15,211420,1


In [ ]:
#Guardamos el archivo
df_user_new= df_user_new.to_csv("df_user_desanidada.csv", index=False)

ETL ARCHIVO steam_games

In [ ]:
#Carga de librerías a trabajar
import gzip
import pandas as pd
import json
import ast
import csv

In [ ]:
#Lectura del archivo json comprimido y conversión a dataframe
with gzip.open("steam_games.json.gz", "rt", encoding="utf-8") as f:
    data = f.readlines()

In [ ]:
data = map(lambda x: x.rstrip(), data)
data_json_str = "[" + ','.join(data) + "]"
df_games = pd.read_json(data_json_str)

In [ ]:
#Se eliminan las columnas que no se necesitan para el procesamiento
df_games = df_games.drop(columns = ['user_id','discount_price','items','steam_id','items_count'])

In [ ]:
#Se eliminan registros con valor nulo en todos los campos
df_games.dropna(how='all', inplace=True)

In [ ]:
#Se convierten en 0 los registros que no tienen id
df_games[id]=df_games['id'].fillna(0).astype(int)
df_games[id]=df_games['id'].fillna(0)

In [ ]:
#Generación de la variable año desde release_date
#Se da formato fecha a los valores de la columna release_date, quedando como nulos los que no tienen el formato
df_games['release_date']=pd.to_datetime(df_games['release_date'], errors='coerce')

#Se asegura que los datos cumplan con el formato AAAA-MM-DD
df_games['release_date']=pd.to_datetime(df_games['release_date']).dt.strftime('%Y-%m-%d')

#Se identifican los valores nulos de la columna release_date y se eliminan
df_games['release_date'].isnull().sum()
df_games = df_games.dropna(subset=['release_date'])

#Se crea la variable year
df_games['release_date']=pd.to_datetime(df_games['release_date'])
df_games['year']=df_games['release_date'].dt.year

#Se verifican los cambios
df_games[['release_date','year']].tail(15)


In [ ]:
#Se transforma la columna price
#Se reemplazan los valores free a 0
df_games['price'] = df_games['price'].replace(['Free to Play','Free Demo','Free HITMANâ„¢ Holiday Pack','Play the Demo','Free Mod','Play WARMACHINE: Tactics Demo','Free To Play','Free'], 0)

In [ ]:
#Se reemplazan los registros de price que noi tiene relación con el precio por NaN
df_games['price'] = df_games['price'].replace(['Install Theme','Third-party','Starting at $','Starting at $','Play Now','Starting at $499.00','Starting at $449.00'], 'NaN')

In [10]:
#Se eliminan los regsitros NaN de price
df_games.dropna(subset=['price'])

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,metascore,developer,year
0,0,0,0,88310,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,04/01/2018,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,['Single-player'],4.99,0,761140.0,NaN,Kotoshiro,2018
1,1,1,1,88311,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,04/01/2018,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",http://steamcommunity.com/app/643980/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",0.00,0,643980.0,NaN,Secret Level SRL,2018
2,2,2,2,88312,Poolians.com,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,24/07/2017,"['Free to Play', 'Simulation', 'Sports', 'Casu...",http://steamcommunity.com/app/670290/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",0.00,0,670290.0,NaN,Poolians.com,2017
3,3,3,3,88313,å½¼å²¸é¢†åŸŸ,"['Action', 'Adventure', 'Casual']",å¼¹ç‚¸äºº2222,å¼¹ç‚¸äºº2222,http://store.steampowered.com/app/767400/2222/,07/12/2017,"['Action', 'Adventure', 'Casual']",http://steamcommunity.com/app/767400/reviews/?...,['Single-player'],0.99,0,767400.0,NaN,å½¼å²¸é¢†åŸŸ,2017
4,4,4,5,88315,Trickjump Games Ltd,"['Action', 'Adventure', 'Simulation']",Battle Royale Trainer,Battle Royale Trainer,http://store.steampowered.com/app/772540/Battl...,04/01/2018,"['Action', 'Adventure', 'Simulation', 'FPS', '...",http://steamcommunity.com/app/772540/reviews/?...,"['Single-player', 'Steam Achievements']",3.99,0,772540.0,NaN,Trickjump Games Ltd,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29778,29778,29778,32129,120439,Bidoniera Games,"['Action', 'Adventure', 'Casual', 'Indie']",Kebab it Up!,Kebab it Up!,http://store.steampowered.com/app/745400/Kebab...,04/01/2018,"['Action', 'Indie', 'Casual', 'Violent', 'Adve...",http://steamcommunity.com/app/745400/reviews/?...,"['Single-player', 'Steam Achievements', 'Steam...",1.99,0,745400.0,NaN,Bidoniera Games,2018
29779,29779,29779,32130,120440,Ghost_RUS Games,"['Casual', 'Indie', 'Simulation', 'Strategy']",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,04/01/2018,"['Strategy', 'Indie', 'Casual', 'Simulation']",http://steamcommunity.com/app/773640/reviews/?...,"['Single-player', 'Steam Achievements']",1.99,0,773640.0,NaN,"Nikita ""Ghost_RUS""",2018
29780,29780,29780,32131,120441,Sacada,"['Casual', 'Indie', 'Strategy']",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,04/01/2018,"['Strategy', 'Indie', 'Casual']",http://steamcommunity.com/app/733530/reviews/?...,"['Single-player', 'Steam Achievements', 'Steam...",4.99,0,733530.0,NaN,Sacada,2018
29781,29781,29781,32132,120442,Laush Studio,"['Indie', 'Racing', 'Simulation']",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,04/01/2018,"['Indie', 'Simulation', 'Racing']",http://steamcommunity.com/app/610660/reviews/?...,"['Single-player', 'Steam Achievements', 'Steam...",1.99,0,610660.0,NaN,Laush Dmitriy Sergeevich,2018


In [11]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29783 entries, 0 to 29782
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0.3  29783 non-null  int64  
 1   Unnamed: 0.2  29783 non-null  int64  
 2   Unnamed: 0.1  29783 non-null  int64  
 3   Unnamed: 0    29783 non-null  int64  
 4   publisher     23774 non-null  object 
 5   genres        28549 non-null  object 
 6   app_name      29782 non-null  object 
 7   title         29782 non-null  object 
 8   url           29783 non-null  object 
 9   release_date  29783 non-null  object 
 10  tags          29622 non-null  object 
 11  reviews_url   29782 non-null  object 
 12  specs         29114 non-null  object 
 13  price         28776 non-null  float64
 14  early_access  29783 non-null  int64  
 15  id            29782 non-null  float64
 16  metascore     2545 non-null   float64
 17  developer     28533 non-null  object 
 18  year          29783 non-nu

In [9]:
#Se convierte a CSV el dataframe
data_df = df_games.to_csv("df_steam_games.csv", index=False)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                